In [ ]:
# !pip install openai==0.28

In [ ]:
import openai

# see: https://platform.openai.com/docs/quickstart
openai.api_key = ""  # add your key here
from IPython.display import display, Markdown
import requests
import json

In [ ]:
# Define the DotDict class
class DotDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __init__(self, dct):
        for key, value in dct.items():
            if hasattr(value, "keys"):
                value = DotDict(value)
            self[key] = value

In [55]:
def clinical_trials_api(condition, terms, intervention, topK=5):
    assert topK <= 10

    # Define the payload for the request
    payload = {
        "query.cond": condition,
        "query.term": terms,
        "query.intr": intervention,
        # "filter.overallStatus": 'COMPLETED',
    }

    # Define the URL and make the request
    url = "https://clinicaltrials.gov/api/v2/studies"
    response = requests.get(url, params=payload)

    trial_ids = []
    clinical_trial_data = []

    # Check if the request was successful
    if response.status_code == 200:
        try:
            data = response.json()
            studies = data.get("studies", [])[:topK]

            for idx, study_data in enumerate(studies):
                study = DotDict(study_data)
                try:
                    # Get the study number (nctId)
                    nct_id = study.protocolSection.identificationModule.nctId
                    trial_ids.append("https://clinicaltrials.gov/study/" + str(nct_id))
                    clinical_trial_data.append(
                        "Clinical Trial Details #{} in Nested JSON format:".format(
                            idx + 1
                        )
                    )
                    clinical_trial_data.append(json.dumps(study))
                except (KeyError, AttributeError):
                    continue

            # Save clinical trial data as a string in results_dict
            clinical_trial_data = "\n".join(clinical_trial_data)

        except requests.exceptions.JSONDecodeError:
            print("Failed to decode JSON. Here is the raw response:")
            print(response.text)
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)

    # Print the collected clinical trial data
    print("The following top {} clinical trials are pulled:".format(topK))
    print("\n".join(trial_ids))
    # print(clinical_trial_data)

    return trial_ids, clinical_trial_data

In [49]:
def chatgpt_summary(context):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # "o1-mini" (more expensive) or "gpt-4o-mini" (cheaper)
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "Please summarize the most clinically relevant information out of these nested JSON responses from a Clinical Trial Database for various related clinical trials. Please start with summary right away without any small talk. This is the trial data: {}".format(
                    context
                ),
            },
        ],
    )
    display(Markdown(response["choices"][0]["message"]["content"]))
    return response

In [50]:
def chatgpt_ask(context, question):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # "o1-mini" (more expensive) or "gpt-4o-mini" (cheaper)
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "Please thoroughly understand clinically important information encoded in the following nested JSON responses from a Clinical Trial Database for various related clinical trials. Please answer the subsequent questions based on this data. Please be factual and always cite the clinical trial source when you use a certain piece of information from there. Here is the trial data: {}".format(
                    context
                ),
            },
            {
                "role": "user",
                "content": "Now answer the following question: {}".format(question),
            },
        ],
    )
    display(Markdown(response["choices"][0]["message"]["content"]))
    return response

# Demo

In [57]:
condition = "infarction"
terms = "diabetes"
intervention = "heart attack"
top_number_of_trials_to_consider = 3

question = """

What are the innovative ideas have been introduced in the clinical trials recently that are quite differentiating from the regular/conventional way 
of executing clinical trials? Do the trials look hopeful at all? Are any of them still recruiting?

"""

In [58]:
clinical_trials_context = clinical_trials_api(
    condition, terms, intervention, topK=top_number_of_trials_to_consider
)

The following top 3 clinical trials are pulled:
https://clinicaltrials.gov/study/NCT05332262
https://clinicaltrials.gov/study/NCT06245980
https://clinicaltrials.gov/study/NCT02542774


In [59]:
response = chatgpt_summary(clinical_trials_context)

### Summary of Clinical Trials

**1. NCT05332262 - A Precision Medicine Approach to Identify Patients Undergoing Elective PCI at Risk of Peri-PCI Myocardial Infarction**
   - **Organization:** University of Florida
   - **Recruitment Status:** Currently recruiting
   - **Study Design:** Prospective observational cohort study
   - **Population:** 500 stable CAD patients undergoing elective PCI with clopidogrel
   - **Objective:** Validate ABCD-GENE score to identify patients at risk for peri-PCI myocardial infarction or myocardial injury
   - **Primary Endpoint:** Rate of peri-PCI myocardial infarction/myocardial injury within 24 hours post-PCI, comparing high vs. low ABCD-GENE score cohorts.
   - **Inclusion Criteria:** Stable CAD patients, age ≥18, troponin negative before PCI.
   - **Exclusion Criteria:** Recent myocardial infarction, treatment with other P2Y12 inhibitors, hypersensitivity to clopidogrel, etc.

**2. NCT06245980 - Role of SGLT2I in Diabetic Patients With Myocardial Infarction**
   - **Organization:** Chongqing Medical University
   - **Recruitment Status:** Currently recruiting
   - **Study Design:** Prospective observational study
   - **Population:** 1200 acute myocardial infarction patients with type 2 diabetes
   - **Objective:** Assess the impact of SGLT2 inhibitors on outcomes in diabetic patients who experienced myocardial infarction
   - **Primary Outcomes:** Major adverse cardiovascular events (MACE) and heart failure readmission within 24 months.
   - **Inclusion Criteria:** Diabetic patients with acute myocardial infarction.
   - **Exclusion Criteria:** Patients undergoing CABG, severe valvular heart disease, contraindications to preventive therapies, prior use of SGLT2i, severe hepatic and renal insufficiency.

**3. NCT02542774 - Sex Differences in the Risk of Cardiovascular Diseases Associated With Diabetes**
   - **Organization:** University College, London
   - **Recruitment Status:** Active, but not recruiting
   - **Study Design:** Retrospective observational cohort study
   - **Population:** Estimated 80,000 patients linked through electronic health records with diabetes
   - **Objective:** Evaluate cardiovascular event risk differences in men and women with diabetes, focusing on medication usage for blood glucose, blood pressure, and cholesterol control.
   - **Primary Outcomes:** Incidence of various cardiovascular diseases over a 10-year follow-up, including myocardial infarction and other related conditions.
   - **Inclusion Criteria:** Patients with diabetes (new-onset or existing), age ≥18 years.

These clinical trials focus on identifying high-risk patients and evaluating treatment methodologies for those experiencing cardiovascular issues, particularly in the context of diabetes and elective PCI procedures.

In [60]:
response = chatgpt_ask(clinical_trials_context, question.strip())

The clinical trials highlighted demonstrate several innovative ideas and approaches that differentiate them from conventional trial methodologies:

1. **Use of Precision Medicine**: 
   - The first trial (NCT05332262) utilizes a precision medicine approach to identify patients at risk of peri-PCI myocardial infarction or myocardial injury. This study integrates genetic data with standard clinical variables to create an ABCD-GENE score, which stratifies patients into different risk cohorts. This method aims to tailor antiplatelet therapies based on individual patient risk profiles, moving away from a one-size-fits-all approach (ClinicalTrials.gov, NCT05332262).

2. **Focus on Genetic Factors in Patient Stratification**: 
   - The incorporation of genetics (specifically the CYP2C19 LOF) alongside clinical factors provides a more individualized approach to patient management in the context of coronary interventions. This prospective validation is essential, as previous scores were developed retrospectively (ClinicalTrials.gov, NCT05332262).

3. **Assessment of Medication Impact in Specific Populations**: 
   - The second trial (NCT06245980) investigates the role of SGLT2 inhibitors in diabetic patients who have experienced a myocardial infarction. This trial aims to observe the medication's specific effects in a defined population, emphasizing tailored therapeutic interventions over generalized findings (ClinicalTrials.gov, NCT06245980).

4. **Utilization of Electronic Health Records for Large-Scale Data Analysis**: 
   - The third trial (NCT02542774) analyzes linked electronic health records from a significant database, CALIBER, to evaluate cardiovascular risk in men and women with diabetes. This observational approach allows for retrospective analysis on a large scale, leveraging existing health data to uncover insights on disease progression and treatment outcomes (ClinicalTrials.gov, NCT02542774).

### Hopefulness of the Trials
- **Hopefulness**: The first trial's innovative approach in identifying high-risk patients could significantly improve outcomes in elective PCI, as it aims to reduce complications through better risk stratification. The focus on diabetic patients in the second trial supports the growing recognition of tailored therapy in chronic disease management, promising to enhance care for this vulnerable population. The third trial aims to address disparities in health service delivery and cardiovascular risk, suggesting the potential for improved outcomes through targeted care.

### Recruitment Status
- **Recruitment Status**: 
  - The first trial (NCT05332262) is currently **recruiting** participants (ClinicalTrials.gov, NCT05332262).
  - The second trial (NCT06245980) is also still **recruiting** individuals (ClinicalTrials.gov, NCT06245980).
  - The third trial (NCT02542774), while originally active, is listed as **unknown** with a last known status of **active not recruiting**, indicating that no new participants may currently be enrolled (ClinicalTrials.gov, NCT02542774).

In summary, these trials demonstrate promising innovations in clinical research design, focusing on personalized medicine, leveraging health data analytics, and aiming for better-targeted treatments. The ongoing recruitment in two of the trials indicates optimism for potential breakthroughs in treatment methodologies.